In [1]:
import numpy as np
import pandas as pd
import MetaTrader5 as mt5


In [15]:
import datetime


In [26]:

def fget_data(currency,num_c):
    mt5.initialize()
    ohlc_data = pd.DataFrame(mt5.copy_rates_from_pos(currency, mt5.TIMEFRAME_M15, 0,num_c))
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data





In [27]:
nowd=fget_data('GOLD',110)
nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
last_row = nowd.iloc[-2]
print(last_row)

time                    1703684700
Open                       2067.18
High                       2068.73
Low                        2067.15
Close                      2068.07
tick_volume                    706
spread                          27
real_volume                      0
mtime          2023-12-27 13:45:00
Name: 108, dtype: object


In [23]:
def fget_data2(currency):
    mt5.initialize()
    ohlc_data = pd.DataFrame(mt5.copy_rates_range(currency, mt5.TIMEFRAME_M15, datetime.datetime(2023, 1, 1), datetime.datetime.now()))
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data

usingdt3=fget_data2('GOLD')

In [29]:

def distance_checker(price,value,lim):
    plh=price-value
    plh=abs(plh)
    if plh >lim:
        return 1
    else:
        return 0
    
def find_status(data,size):
    data['AbsDiff'] = abs(data['Close'] - data['Open'])

# Find the maximum value among the calculated differences
    max_diff = data['AbsDiff'].max()
    if max_diff>size:
        return 1
    else:
        return 0
    

In [46]:
from schedule import every, repeat, run_pending

In [50]:
# print(last_row['mtime'].time())


def daily_start():
    nowd=fget_data('GOLD',110)
    nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
    last_row = nowd.iloc[-2]
    print('yes')
    pg=last_row['mtime'].time()
    if not datetime.time(0, 0) <= pg <= datetime.time(3, 0):
        pos_info = mt5.positions_total()

        # Check if there are open positions or active orders
        if pos_info==0:
            target_date = last_row['mtime'].date()

        # Filter the DataFrame for the specific day
            day_data = nowd[nowd['mtime'].dt.date == target_date]

        # Find the maximum and minimum points for the day
            max_price = day_data['High'].max()
            min_price = day_data['Low'].min()
            price=last_row['Close']
            # print(price)
            if price>max_price:
                if distance_checker(price,max_price,1.5)==0:
                    max_price=price
                elif distance_checker(price,max_price,1.5)==1:
                    win_data=nowd.tail(11)
                    if find_status(win_data,1.5):
                        lowp=last_row['Low']
                        sl=lowp-1.5
                        expectedloss=price-sl
                        target_tp=expectedloss*5
                        tp=price+target_tp
                        lot=0.1
                        deviation=0.3
                        request = {"action": mt5.TRADE_ACTION_DEAL,"symbol": 'GOLD',"volume": lot,"type": mt5.ORDER_TYPE_BUY,"price": price,"sl": sl,"tp": tp,"deviation": deviation,"magic": 2002,"comment": "python script open","type_time": mt5.ORDER_TIME_GTC,"type_filling": mt5.ORDER_FILLING_RETURN,}
                        mt5.order_send(request)
                        
            elif price<min_price:
                if distance_checker(price,min_price,1.5)==0:
                    min_price=price
                elif distance_checker(price,min_price,1.5)==1:
                    win_data=nowd.tail(11)
                    if find_status(win_data,1.5):
                        highp=last_row['High']
                        sl=highp+1.5
                        expectedloss=sl-price
                        target_tp=expectedloss*5
                        tp=price-target_tp
                        lot=0.1
                        deviation=0.3
                        request = {"action": mt5.TRADE_ACTION_DEAL,"symbol": 'GOLD',"volume": lot,"type": mt5.ORDER_TYPE_SELL,"price": price,"sl": sl,"tp": tp,"deviation": deviation,"magic": 2002,"comment": "python script open","type_time": mt5.ORDER_TIME_GTC,"type_filling": mt5.ORDER_FILLING_RETURN,}
                        mt5.order_send(request)
                        
                    
            
        

      

In [40]:
import time as tt

In [38]:
import schedule


In [51]:
daily_start()

yes


In [49]:
run_pending()

In [67]:
maxholder=0
minholder=0
def get_first_values():
    global maxholder
    global minholder
    nowd=fget_data('GOLD',110)
    nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
    last_row = nowd.iloc[-2]
    target_date = last_row['mtime'].date()

    # Filter the DataFrame for the specific day
    day_data = nowd[nowd['mtime'].dt.date == target_date]

    # Find the maximum and minimum points for the day
    max_price = day_data['High'].max()
    min_price = day_data['Low'].min()
    maxholder = max_price
    minholder = min_price
    
get_first_values()
print(maxholder)
print(minholder)

2088.31
2071.18


In [66]:
nowd=fget_data('GOLD',110)
nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
last_row = nowd.iloc[-2]
price=last_row['Close']
lowp=2071
sl=lowp-11.50
expectedloss=price-sl
target_tp=expectedloss*5
tp=price+target_tp
lot=0.1
deviation=1
request = {"action": mt5.TRADE_ACTION_DEAL,"symbol": 'GOLD',"volume": lot,"type": mt5.ORDER_TYPE_BUY,"price": price,"sl": sl,"tp": tp,"deviation": deviation,"magic": 2002,"comment": "python script open","type_time": mt5.ORDER_TIME_GTC,"type_filling": mt5.ORDER_FILLING_IOC,}
mt5.order_send(request)

OrderSendResult(retcode=10009, deal=324714225, order=327667653, volume=0.1, price=2073.91, bid=2073.53, ask=2073.91, comment='Request executed', request_id=3094555137, retcode_external=0, request=TradeRequest(action=1, magic=2002, order=0, symbol='GOLD', volume=0.1, price=2073.62, stoplimit=0.0, sl=2059.5, tp=2144.2199999999993, deviation=1, type=0, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))

In [56]:
schedule.every(15).minutes.do(daily_start)

# Run the script indefinitely
while True:
    schedule.run_pending()
    tt.sleep(1)